Data Reading

In [0]:
df = spark.read.table("dtbproject.silver.orders")

In [0]:
df.display()

In [0]:
df_dimproducts = spark.read.table("dtbproject.gold.dimcustomer")

In [0]:
df_dimcustomers = spark.sql("""
SELECT DimCustomerkey, customer_id as dim_customer_id from dtbproject.gold.dimcustomer""")

df_dimproducts = spark.sql(
    """
    SELECT product_id as DimProductKey, product_id as dimproductid FROM dtbproject.gold.dimproducts"""
)

In [0]:
df_dimcustomers.display()
df_dimproducts.display()

In [0]:
df_fact = df.join(df_dimcustomers, df.customer_id == df_dimcustomers.dim_customer_id, 'left')
df_fact = df_fact.join(df_dimproducts, df.product_id == df_dimproducts.dimproductid, 'left')


In [0]:
df_fact.display()

df_fact_new = df_fact.drop("dim_customer_id", "dimproductid", "customer_id")


In [0]:
df_fact_new = df_fact_new.drop("product_id")

In [0]:
df_fact_new.display()

UPSERT ON FACT TABLE

In [0]:
from delta.tables import DeltaTable

In [0]:
if spark.catalog.tableExists("dtbproject.gold.fact_orders"):
    dlt_obj = DeltaTable.forName(spark, "dtbproject.gold.fact_orders")
    dlt_obj.alias("trg").merge(df_fact_new.alias("src"),"trg.order_id = src.order_id AND trg.DimCustomerKey = src.DimCustomerKey")
else:
    df_fact_new.write.format("delta").mode("append").option("path","abfss://gold@azuredatabricksjesslake.dfs.core.windows.net/fact_orders").saveAsTable("dtbproject.gold.fact_orders")

In [0]:
%sql
SELECT * FROM dtbproject.gold.fact_orders

In [0]:
df = spark.read.table("dtbproject.gold.dimproducts")
df.write.format("delta").mode("append").save("abfss://gold@azuredatabricksjesslake.dfs.core.windows.net/dimproducts")